In [1]:
%load_ext nbpipes

In [2]:
import numpy as np

In [3]:
inp = """
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
""".strip().splitlines()

In [4]:
with open("10.txt", "r") as f:
    inp = f.read().strip().splitlines()

In [5]:
indicators = inp |> map[$.split(" ", 1)[0][1:-1] |> map[{".": "0", "#": "1"}[$]] |> list |> reversed |> "".join($) |> int($, 2)]

In [6]:
b_to_js = inp |> map[$.split(" ", 1)[1].rsplit(" ", 1)[0].split() |> map[$[1:-1].split(",") |> map[int]]]
max_buttons = b_to_js |> map[len] |> max
buttons = b_to_js |> map[map[sum . map[1 << $]]]

In [7]:
joltages = inp |> map[$.rsplit(" ", 1)[1][1:-1].split(",") |> map[int]]

In [8]:
def bfs(start, buttons):
    seen = {start}
    q = [(start, 0)]
    while q:
        cur, steps = q.pop(0)
        if cur == 0:
            return steps
        for b in buttons:
            press = cur ^ b
            if press == 0:
                return steps + 1
            elif press in seen:
                continue
            seen.add(press)
            q.append((press, steps + 1))
    raise ValueError("unreachable")

In [9]:
# part 1
ans = 0
for i, b in zip(indicators, buttons):
    ans += bfs(i, b)
ans

535

In [10]:
# part 2
import z3

ans = 0
bvars = [z3.Int(f"x_{i}") for i in range(max_buttons)]
for b_to_j, js in zip(b_to_js, joltages):
    s = z3.Optimize()
    obj = sum(bvars[:len(b_to_j)])
    s.minimize(obj)
    s.add([jval == sum([bvars[b] for b in range(len(b_to_j)) if j in b_to_j[b]]) for j, jval in enumerate(js)])
    s.add([bvar >= 0 for bvar in bvars[:len(b_to_j)]])
    assert s.check() == z3.sat
    ans += s.model().eval(obj).py_value()
ans

21021